# Distributed Inference and Serving

Common distributed inference strategies according to your hardware resources:
- Single GPU
    - No distributed inference.
- Single-node, multi-GPU
    - tensor parallel: model is too large to fit in one GPU but fit in multiple GPUs within a single node.
        - tensor parallel size = number of GPUs
- Multi-node, multi-GPU
    - tensor parallel + pipeline parallel
        - tensor parallel size = number of GPUs per node
        - pipeline parallel size = number of nodes

## Inference memory calculation

$$ M = \frac{P * 4}{32/Q} * 1.2 (GB) $$

- $M$: GPU memory
- $P$: parameters (in Billions, 7B is 7)
- $4$: 4 bytes for each parameter
- $32$: 1 byte is 8 bits, so 4 bytes is 32 bits
- $Q$: quantization bits (e.g., 16 bits, 8 bits, 4 bits)
- $1.2$: 20% overhead

Memory for loading 70B model at 16 bit precision is

$$ \frac{70 * 4}{32/16} * 1.2 = 168GB $$

Tool: [Can you run it](https://huggingface.co/spaces/Vokturz/can-it-run-llm)

A `Standard_NC80adis_H100_v5` AML VM has 2 H100 GPUs with 94 GiB vRAM each (188 GiB together), so using tensor parallel it should be possible to run inference on a Llama 70B model at 16 bit precision.

## Inference Llama-3.3-70B-Instruct

The VM has 2 GPUs, so we run a distributed inference vllm server with `tensor-parallel-size 2`.

To serve a new model vLLM will first download it to a local folder. By default it uses the `/home/.cache/huggingface` folder in Linux. In AML, this folder is in the OS disk, which is only 120G and cannot hold a Llama 70B model. So when running `vllm serve` we specify the `--download-dir` to a different folder created in the mounted fileshare, which is 100TiB. 

```bash
$ vllm serve unsloth/Llama-3.3-70B-Instruct \
$   --tensor-parallel-size 2 \
$   --download-dir "./models/huggingface"
```

At start vLLM will log its args, then download the model (in this particular case, as 30 5G `.safetensors` files, downloaded in parallel).

```bash
INFO 12-08 09:28:05 api_server.py:585] vLLM API server version 0.6.4.post1
INFO 12-08 09:28:05 api_server.py:586] args: Namespace(subparser='serve', model_tag='unsloth/Llama-3.3-70B-Instruct', config='', host=None, port=8000, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, lora_modules=None, prompt_adapters=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=False, disable_frontend_multiprocessing=False, enable_auto_tool_choice=False, tool_call_parser=None, tool_parser_plugin='', model='unsloth/Llama-3.3-70B-Instruct', task='auto', tokenizer=None, skip_tokenizer_init=False, revision=None, code_revision=None, tokenizer_revision=None, tokenizer_mode='auto', chat_template_text_format='string', trust_remote_code=False, allowed_local_media_path=None, download_dir='./models/huggingface', load_format='auto', config_format=<ConfigFormat.AUTO: 'auto'>, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, max_model_len=None, guided_decoding_backend='outlines', distributed_executor_backend=None, worker_use_ray=False, pipeline_parallel_size=1, tensor_parallel_size=2, max_parallel_loading_workers=None, ray_workers_use_nsight=False, block_size=16, enable_prefix_caching=False, disable_sliding_window=False, use_v2_block_manager=False, num_lookahead_slots=0, seed=0, swap_space=4, cpu_offload_gb=0, gpu_memory_utilization=0.9, num_gpu_blocks_override=None, max_num_batched_tokens=None, max_num_seqs=256, max_logprobs=20, disable_log_stats=False, quantization=None, rope_scaling=None, rope_theta=None, hf_overrides=None, enforce_eager=False, max_seq_len_to_capture=8192, disable_custom_all_reduce=False, tokenizer_pool_size=0, tokenizer_pool_type='ray', tokenizer_pool_extra_config=None, limit_mm_per_prompt=None, mm_processor_kwargs=None, enable_lora=False, enable_lora_bias=False, max_loras=1, max_lora_rank=16, lora_extra_vocab_size=256, lora_dtype='auto', long_lora_scaling_factors=None, max_cpu_loras=None, fully_sharded_loras=False, enable_prompt_adapter=False, max_prompt_adapters=1, max_prompt_adapter_token=0, device='auto', num_scheduler_steps=1, multi_step_stream_outputs=True, scheduler_delay_factor=0.0, enable_chunked_prefill=None, speculative_model=None, speculative_model_quantization=None, num_speculative_tokens=None, speculative_disable_mqa_scorer=False, speculative_draft_tensor_parallel_size=None, speculative_max_model_len=None, speculative_disable_by_batch_size=None, ngram_prompt_lookup_max=None, ngram_prompt_lookup_min=None, spec_decoding_acceptance_method='rejection_sampler', typical_acceptance_sampler_posterior_threshold=None, typical_acceptance_sampler_posterior_alpha=None, disable_logprobs_during_spec_decoding=None, model_loader_extra_config=None, ignore_patterns=[], preemption_mode=None, served_model_name=None, qlora_adapter_name_or_path=None, otlp_traces_endpoint=None, collect_detailed_traces=None, disable_async_output_proc=False, scheduling_policy='fcfs', override_neuron_config=None, override_pooler_config=None, disable_log_requests=False, max_log_len=None, disable_fastapi_docs=False, enable_prompt_tokens_details=False, dispatch_function=<function serve at 0x7fc3b956dc60>)
```